### 网络结构

+ 全连接神经网络
+ 卷积神经网络
+ 循环神经网络

### 卷积神经网络和全连接神经网络的差异

+ 全连接神经网络和卷积神经网络的结构直观上差异很大, 但它们的整体架构是相似的,　卷积神经网络的每一个节点都是一个神经元,　
+ 全连接神经网络,　没相邻两层之间的节点都有边相连,　于是会把每一层全连接层中的节点组织成一列，方便显示结构
+ 卷积神经网络相邻两层之间只有部分节点相连,　为了展示一层神经元的维度, 一般会将每一层的节点组织成三维矩阵
+ 卷积神经网络的输入输出以及训练流程也和全连接神经网络基本一致
+ 卷积神经网络的输入层就是图片的原始像素矩阵, 输出层的每一个节点表示不同类别的可信度,损失函数和以及参数的优化过程也和全连接神经网络相同
+ 最主要的区别是两层之间的连接方式不同

### 全连接神经网络的缺点

+ 全连接神经网络的参数太多
+ 参数过多很容造成计算速度减慢, 过度拟合

### 卷积神经网络的架构

卷积神经网络的前几层中每一个节点只和上一层的部分节点相连接, 

+ 输入层: 神经网络的输入, 输入的是三维矩阵,图片的像素矩阵
+ 卷基层: 卷积层中每一个节点的输入只是在上一层神经网络的一小块, 卷积层尝试将神经网络的一小块进行深入的研究分析, 从而得到抽象成都更高的特征
+ 池化层: 池化层不会改变神经网络的深度,　但是它可以缩小矩阵的大小, 操作可以认为是将一张高分辨率的照片转化为低分辨率的照片,　可以进而缩小参数的个数
+ 全连接层:经过多轮的卷积层和池化层处理,　也就是使用卷积层和池化层完成更高特征的提取,　再经过全连解神经网络完成分类任务
+ softmax层: softmax 主要应用于分类任务,　得到当前样例的不同概率分布情况

### 卷积神经网络的常用结构

+ 过滤器可以将当前层神经网络上的一个子节点矩阵转化为下一层神经网络上的一个单位节点矩阵, 单位节点矩阵是指的是长度和宽度都是 1, 但深度不限的节点矩阵
+ 过滤器的尺寸: 输入节点矩阵的大小, 而深度指的是输出单位节点矩阵的深度;
+ 卷积层结构的前向传播过程就是将一个过滤器从神经网络当前层的左上角移动到右下角;并且移动中计算每一个对应单位矩阵得到的;
+ 首先将这个过滤器用于左上角子矩阵,　然后移动到左下, 右上、右下、矩阵，每一移动一次 会计算出一个值, 这些值拼接成新的矩阵, 完成卷积层前向传播过程, 
+ 在当前层矩阵的边界上加入全 0 填充, 这样可以保证卷积层前向传播结果矩阵的大小和当前层矩阵保持一致
+ 除了使用全 0 填充还可以设置过滤器移动的步长来调整结果矩阵的大小, 
+ 参数的设定, 每一个卷积层中使用的过滤器中的参数都是一样的, 共享过滤的参数可以使得图像上的内容不受位置的影响,　共享

In [ ]:
import tensorflow as tf

# 通过 tf.get_variable 的方式创建过滤器的权重变量和偏置变量, 上面介绍的卷积层的参数个数只和过滤的的尺寸深度以及当前层
# 节点矩阵的深度有关 所以这里声明的参数变量是一个四维矩阵,　前面两个维度代表过滤器的尺寸,　第三个维度表示当前层的深度,
# 第四维表示过滤器的深度

# 过滤器权重
filter_weight = tf.get_variable('weights',[5,5,3,16],initializer=tf.turncated_normal_initializer(stddev=0.1))

# 过滤器的偏置
biases = tf.get_variable('biases',[16],initializer=tf.constant_initializer(0.1))

# 实现卷积的前向神经算法
conv = tf.nn.conv2d(input,filter_weight,strides=[1,1,1,1],padding='SAME')

# 每个节点加上偏置项,　
bias = tf.nn.bias_add(conv,biases)

# 将计算结果通过 ReLU 激活函数去线性化
actived_conv = tf.nn.relu(bias)

### 池化层

+ 池化层非常有效地缩小矩阵的尺寸, 从而减少最后全连接层中的参数, 池化层既可以加快计算速度也可以防止过度拟合问题的作用
+ 池化层前向传播的过程也是通过类过滤器的结构完成的, 不过池化层过滤器中计算不是节点的加权和, 而是采用更简单的最大值或平均值运算, 
+ 最大池化层, 使用最大值操作,使用最多的池化层
+ 最小池化层, 使用最小值操作,使用较少的池化层

+ 池化层的过滤器也需要人工设定过滤器的尺寸, 是否使用全 0 填充以及过滤器移动步长等设置, 唯一的区别在于卷积层使用的过滤器是横跨整个深度, 而池化层使用的过滤只影响一个深度上的节点, 所以池化层除了长度和宽度之外还需要在深度这个维度上移动, 

In [ ]:
# 池化层的前向传播过程, 采取 tf.nn.max_pool 实现最大池化层; 它的参数 和 tf.nn.conv2d 类似

## ksize 提供了过滤器的尺寸, strides 提供了步长信息, padding 提供了是否使用了全 0 填充, 
tf.nn.max_pool(actived_conv,kisze=[1,3,3,1],strides=[1,2,2,1],padding='SAME')